<a href="https://colab.research.google.com/github/nrohrbach/PocDatenkatalogBafu/blob/main/PoC_Datenkatalog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proof of Concept Datenkatalog BAFU für neue Webseite auf Livingdocs
Auf der BAFU Webseite sollen künftig alle Daten, Indikatoren, Statistiken und Geodatenmodelle über einen zentralen Katalog gefunden werden. Die Inhalte des Katalogs stammen aus der API von opendata.swiss und von anderen Quellen.

In [1]:
import requests
import pandas as pd


In [67]:
# Filtermöglichkeiten des Katalogs vorbereiten

Thema =   ['Abfall',
           'Altlasten',
           'Bildung, Forschung, Innovation',
           'Biodiversität',
           'Biotechnologie',
           'Boden',
           'Chemikalien',
           'Elektrosmog und Licht',
           'Ernährung, Wohnen, Mobilität',
           'Gesundheit',
           'Internationales',
           'Klima',
           'Landschaft',
           'Lärm',
           'Luft',
           'Naturgefahren',
           'Recht',
           'Störfallvorsorge',
           'Umweltverträglichkeitsprüfung',
           'Wald & Holz',
           'Wasser',
           'Wirtschaft und Konsum'
           ]

Datentyp = ['Indikator',
            'Statistik',
            'Geodaten',
            'Geodatenmodell',
            'Monitoring',
            'Daten']

Monitoring = ['LFI',
              'NABO',
              'NABEL',
              'NADUF',
              'NAWA',
              'NAQUA',
              'LABES',
              'BDM',
              'SonBase',]

In [3]:
# Mapping der BAFU-Themen auf das Keyword bei opendata.swiss
def map_options(option):
  mapping = {
      'Abfall': 'abfall',
      'Altlasten': 'altlasten',
      'Bildung, Forschung, Innovation': 'bildung',
      'Biodiversität': 'biodiversitat',
      'Biotechnologie' : 'biotechnologie',
      'Boden' : 'boden',
      'Chemikalien' : 'chemikalien',
      'Elektrosmog und Licht' : 'elektrosmog',
      'Ernährung, Wohnen, Mobilität'  : 'ernaehrung',
      'Gesundheit'  : 'gesundheit',
      'Internationales'  : 'internationales',
      'Klima'  : 'klima',
      'Landschaft'  : 'landschaft',
      'Lärm'  : 'laerm',
      'Luft'  : 'luft',
      'Naturgefahren'  : 'naturgefahren',
      'Recht'  : 'recht',
      'Störfallvorsorge'  : 'storfallvorsorge',
      'Umweltverträglichkeitsprüfung'  : 'umweltvertraeglichkeitpruefung',
      'Wald & Holz'  : 'wald',
      'Wasser'  : 'wasser',
      'Wirtschaft und Konsum'  : 'wirtschaft'
  }
  return mapping.get(option, None) # Return None if option is not found


In [97]:
url = "https://opendata.swiss/api/3/action/package_search"
params = {
    "q": "organization:bundesamt-fur-umwelt-bafu",
    "rows": 1000  # Request a large number of rows to get all entries
}

response = requests.get(url, params=params)

if response.status_code == 200:
    data = response.json()
    if data['success']:
        packages = data['result']['results']
        df = pd.DataFrame(packages)
    else:
        print("API request was not successful.")
        print(data['error'])
else:
    print(f"Error: Failed to retrieve data. Status code: {response.status_code}")

Found 342 entries for organization=bundesamt-fur-umwelt-bafu


In [104]:
dfOpendataSwiss = df
dfOpendataSwiss = dfOpendataSwiss[['keywords', 'title','description', 'modified']]
dfOpendataSwiss['description'] = dfOpendataSwiss['description'].apply(lambda x: x['de'] if isinstance(x, dict) and 'de' in x else None)
dfOpendataSwiss['title'] = dfOpendataSwiss['title'].apply(lambda x: x['de'] if isinstance(x, dict) and 'de' in x else None)
dfOpendataSwiss['Typ'] = 'Daten'
dfOpendataSwiss['Typ'] = dfOpendataSwiss.apply(lambda row: 'Geodatenmodell' if 'Geodatenmodell' in str(row['title']) else row['Typ'], axis=1)

# prompt: wenn ein Wort aus dem Array Monitoring in title oder description vorkommt, wird das Attribut Typ auf Monitoring gesetzt
dfOpendataSwiss['Typ'] = dfOpendataSwiss.apply(
    lambda row: 'Monitoring' if any(word in str(row['title']) or word in str(row['description']) for word in Monitoring) else row['Typ'],
    axis=1
)

/tmp/ipython-input-104-2076541339.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfOpendataSwiss['description'] = dfOpendataSwiss['description'].apply(lambda x: x['de'] if isinstance(x, dict) and 'de' in x else None)
/tmp/ipython-input-104-2076541339.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfOpendataSwiss['title'] = dfOpendataSwiss['title'].apply(lambda x: x['de'] if isinstance(x, dict) and 'de' in x else None)
/tmp/ipython-input-104-2076541339.py:4: SettingWithCopyWarning: 
A value is tryi

In [107]:
# Statistiken und Indikatoren lesen
dfStatistikenIndikatoren = pd.read_excel("StatistikenIndikatoren.xlsx")

# Mapping auf Keywords
dfStatistikenIndikatoren['keywords'] = dfStatistikenIndikatoren['keywords'].apply(lambda x: [map_options(x)] if pd.notnull(x) and map_options(x) is not None else [])


In [111]:
# Combine the two dataframes
dfCombined = pd.concat([dfOpendataSwiss, dfStatistikenIndikatoren], ignore_index=True)

In [4]:
# Funktion fragt alle Datensätze von opendata.swiss nach einem Keyword ab

def search_datasets(keyword):
    # Basis-URL der CKAN API
    base_url = "https://opendata.swiss/api/3/action/package_search"

    # Parameter für die Abfrage
    params = {
        "fq": f"organization:{'bundesamt-fur-umwelt-bafu'}",
        "q": f"keywords_de:{keyword}"
    }

    # Anfrage an die API senden
    response = requests.get(base_url, params=params)
    data = response.json()
    data = data["result"]["results"]

    # Daten aus Json auslesen
    description = [s['description']['de'] for s in data]
    title = [s['title']['de'] for s in data]
    url0 = [s['resources'][0]['url'] for s in data]
    title0 = [s['resources'][0]['title']['de'] for s in data]
    url1 = [s['resources'][1]['url'] for s in data]
    title1 = [s['resources'][1]['title']['de'] for s in data]

    # Daten als Dataframe speichern
    dict = {'title': title, 'description': description, 'Link': url0}
    dfOpendataSwiss = pd.DataFrame(dict)

    return dfOpendataSwiss

In [5]:
search_datasets('wald')

,title,description,Link
0,LFI: Themen nach NaiS-Waldformationen (10 Klas...,Das Landesforstinventar (LFI) ist eine periodi...,https://visualize.admin.ch/en/browse?previous=...
1,LFI: Themen nach Baumzustand,Das Landesforstinventar (LFI) ist eine periodi...,https://lindas.admin.ch/sparql/#query=PREFIX%2...
2,LFI: Themen nach Anzahl Schäden,Das Landesforstinventar (LFI) ist eine periodi...,https://visualize.admin.ch/en/browse?previous=...
3,LFI: Themen nach Baumzustand (stehend/liegend),Das Landesforstinventar (LFI) ist eine periodi...,https://lindas.admin.ch/sparql/#query=PREFIX%2...
4,LFI: Themen nach Schafthöhe gebrochener Dürrst...,Das Landesforstinventar (LFI) ist eine periodi...,https://lindas.admin.ch/sparql/#query=%23pragm...
5,LFI: Themen nach Zeitpunkt des letzten Eingrif...,Das Landesforstinventar (LFI) ist eine periodi...,https://visualize.admin.ch/en/browse?previous=...
6,LFI: Themen nach Baumart (56 Klassen),Das Landesforstinventar (LFI) ist eine periodi...,https://lindas.admin.ch/sparql/#query=PREFIX%2...
7,LFI: Themen nach eingeführte Baumart (baumförm...,Das Landesforstinventar (LFI) ist eine periodi...,https://lindas.admin.ch/sparql/#query=PREFIX%2...
8,LFI: Themen nach Eigentum (2 Klassen),Das Landesforstinventar (LFI) ist eine periodi...,https://visualize.admin.ch/en/browse?previous=...
9,LFI: Themen nach Brusthöhendurchmesser (10 Kla...,Das Landesforstinventar (LFI) ist eine periodi...,https://lindas.admin.ch/sparql/#query=PREFIX%2...
